<a href="https://colab.research.google.com/github/MatheusPiassiC/redes_neurais/blob/main/keras_implementation_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Estudo do framework Keras!
O objetivo deste estudo é, usando como base os notebooks feitos pelo professor Denilson Alves Pereira, criar uma rede neural para classificar as flores da base de dados Iris.

A base de dados [Iris](https://www.kaggle.com/datasets/uciml/iris) contém dados de 150 flores (50 de cada), em que cada flor tem os atributos:

- Id
- SepalLengthCm
- SepalWidthCm
- PetalLengthCm
- PetalWidthCm
- Species

O objetivo será usar uma rede neural para classificar um determinado conjunto de treino em uma das três espécies possíveis (*Iris-Setosa, Iris-Versicolor, Iris-Virginica*).

In [202]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

##Dados para treino
Fonte: https://www.kaggle.com/datasets/uciml/iris/data

Neste caso, estou armazenando meu dataset no Google Drive para facilitar, mas é bastante simples mudar a localização do arquivo, já que consiste apenas em uma importação de .csv simples.

In [203]:
from google.colab import drive

drive.mount('/content/drive')
caminho = '/content/drive/MyDrive/Colab Notebooks/datasets/Iris.csv'
df = pd.read_csv(caminho)
df.drop(columns=['Id'], inplace=True)
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0            5.1           3.5            1.4           0.2  Iris-setosa
1            4.9           3.0            1.4           0.2  Iris-setosa
2            4.7           3.2            1.3           0.2  Iris-setosa
3            4.6           3.1            1.5           0.2  Iris-setosa
4            5.0           3.6            1.4           0.2  Iris-setosa


Com o dataframe criado e sem a coluna "Id", o próximo passo é separar os nosso conjuntos de treino de de teste, usando a função *train_test_split()*.

In [204]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Species'])
y = df['Species']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 30)

Em seguida, precisamos de normalizar os dados para permitir que o modelo seja treinado da forma mais adequada possível, fazendo com que todos os atributos estejam na mesma escala.

Algo interessante de se observar, é que devemos usar os mesmos parâmetros para escalar tanto o conjunto de treino quanto o conjunto de teste. Por isso, usamos *fit_transform* para escalar os dados de treino, e apenas *transform* para escalar os dados de teste, pois *transform* usa os parâmetros calculados em *fit_transform*.

In [205]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# The boolean feature does not need to be normalized.

X_train = scaler.fit_transform(X_train) #escalaona os dados e calcula os parâmetros necessários pra isso
X_test = scaler.transform(X_test) #usa os parâmetros calculados anteriormente para escalar os dados de teste

Diferente da implementação do neurônio feita anteriormente, o framework *Keras* usa as linhas para representar as instâncias, e as colunas para representar os atributos. Como estamos usando o dataframe *Iris* e retiramos a coluna *ID*, teremos 4 colunas no total.

In [206]:
n = X_train.shape[1] #número de atributos
m = X_train.shape[0] #número de exemplos

print ("Número de atributos: n = " + str(n))
print ("Número de exemplos de treino: m = " + str(m))
print ("Forma do X_train: " + str(X_train.shape))
print ("Forma do y_train: " + str(y_train.shape))
print ("Forma do X_test: " + str(X_test.shape))
print ("Forma do y_test: " + str(y_test.shape))

Número de atributos: n = 4
Número de exemplos de treino: m = 105
Forma do X_train: (105, 4)
Forma do y_train: (105,)
Forma do X_test: (45, 4)
Forma do y_test: (45,)


Um detalhe importante, é que o Keras só aceita números em suas entradas, então é necessário transformar as labels, que antes eram strings, para números.

In [207]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_int = encoder.fit_transform(y_train)   # converte para inteiros
y_test_int  = encoder.transform(y_test)        # usa o mesmo mapeamento

print("y_train_int: ", y_train_int)
print("y_test_int: ", y_test_int)

y_train_int:  [1 1 1 1 1 0 1 2 1 0 1 0 1 1 1 0 1 0 1 0 2 2 2 0 2 1 2 0 0 1 0 2 2 2 2 0 1
 0 1 1 1 2 0 1 0 1 2 1 0 0 0 2 2 0 1 1 1 0 0 0 0 2 0 2 0 0 2 2 2 0 2 1 1 2
 1 1 1 1 2 0 1 0 1 0 0 0 2 1 0 0 1 2 2 2 2 0 1 2 0 2 1 2 2 0 0]
y_test_int:  [0 0 0 2 1 1 2 2 1 2 0 2 1 1 0 1 0 0 0 1 2 0 0 0 2 2 1 2 0 1 2 1 2 2 2 2 1
 2 1 2 2 2 0 1 2]


##Definição do modelo
Apesar de os exemplos iniciais da matéria usarem uma implementação com apenas um neurônio, vou usar uma rede com mais camadas, em que a quantidade de camadas e de neurônios em cada uma delas será definido arbitrariamente.

In [208]:
inputs = keras.Input(shape=(X_train.shape[1],)) #a entrada, no caso, 4 atributos
hidden1 = keras.layers.Dense(10, activation='relu')(inputs) #camada que recebe a entrada, com função de ativação relu
hidden2 = keras.layers.Dense(20, activation='relu')(hidden1) #camada que recebe as saidas da primeira camada, e também usa relu
outputs = keras.layers.Dense(10, activation='softmax')(hidden2) #camada de saída, usa a função de ativação softmax para determinar a saída final
model = keras.Model(inputs=inputs, outputs=outputs) #cria o modelo de fato

In [209]:
model.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 10)             │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 20)             │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 10)             │           210 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 480 (1.88 KB)

 Trainable params: 480 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

##Compilação do modelo
Para a compilação de um modelo, a biblioteca deve receber alguns parâmetros para escolher qual a melhor forma de representar o modelo em hardware. O objetivo da função *compile* é basicamente configurar o processo de treino do modelo. Ele não treina nada por si só, mas configura:
- Um otimizador para o modelo;
- Define a função de perda;
- Cria objetos de métrica;
- Define como a função de treino será executada

Mais adiante, quando chamarmo a função *model.fit*, o Keras usará essas configurações para treinar o modelo, repetindo os passos: **forward → loss → backward → update**

In [210]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

##Treinamento do modelo
É aqui que treinaremos o modelo para os dados de treino fornecidos (X_train e y_train). Para isso, devemos informar os dados de treino e o número de *epochs*(basicamente, iterações). Uma *epoch* é definida por uma iteração por todos os conjuntos de treino.

In [211]:
history = model.fit(X_train, y_train_int, epochs=100)

print(history.history)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0038 - loss: 2.5065   
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0638 - loss: 2.3636 
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0982 - loss: 2.2290 
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1693 - loss: 2.1523 
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.2889 - loss: 2.0694 
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4593 - loss: 1.9711 
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4898 - loss: 1.8996
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5227 - loss: 1.8402 
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5546 - loss: 1.7455 
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5386 - loss: 1.7026 
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6205 - loss: 1.6177 
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6181 - l

##Avaliação do modelo
Agora, com o nosso modelo treinado, podemos avalia-lo com o nosso conjunto de teste

In [212]:
loss, acc = model.evaluate(X_test, y_test_int)
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9495 - loss: 0.1790 
loss: 0.18
acc: 0.96


In [213]:
predictions = model.predict(X_test)

# predições finais (classe mais provável para cada exemplo)
y_pred = np.argmax(predictions, axis=1)

# valores corretos
y_true = y_test_int

print("Predições: ", y_pred)
print("Corretos:  ", y_true)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predições:  [0 0 0 2 1 1 2 2 1 2 0 2 1 1 0 1 0 0 0 2 2 0 0 0 2 2 2 2 0 1 2 1 2 2 2 2 1
 2 1 2 2 2 0 1 2]
Corretos:   [0 0 0 2 1 1 2 2 1 2 0 2 1 1 0 1 0 0 0 1 2 0 0 0 2 2 1 2 0 1 2 1 2 2 2 2 1
 2 1 2 2 2 0 1 2]
